# Säljrapport

*Förslag på rapportstruktur*

## Introduktion

Här skriver vi en kort introduktion. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from src.io_utils import load_data, clean_data, save_data

# Load
df_raw = load_data("data/ecommerce_sales.csv")

# Clean
df_clean = clean_data(df_raw)

# Save
save_data(df_clean, "data/clean_data.csv")

df_clean.head()

## Nyckeltal

Här visualiserar vi våra nyckeltal.

### 1. Vad säljer? – vilka kategorier driver mest intäkt?

**Nyckeltal:** Total intäkt och totalt antal enheter & Intäkt per kategori

### 2. Var säljer vi? – vilka städer står för störst intäkt?

**Nyckeltal:** Intäkt per stad 

In [ ]:
from src.metrics import revenue_per_city, top3_cities
from src.viz import plot_tot_revenue_per_city

print("Top 3 cities per sales:")
print(top3_cities(df_clean))

print()
plot_tot_revenue_per_city(df_clean)

### 3. När säljer vi? – finns tidsmönster/säsong i försäljningen?

 **Nyckeltal:** TBD

In [ ]:
from src.metrics  import  revenue_over_time

#revenue_over_time(df_clean)

ts_month = revenue_over_time(df_clean)
ts_month.head()



In [4]:
from src import metrics as M
from src.viz import revenue_monthly_bar

revenue_total = M.revenue_total(df_clean)
revenue_by_month = M.revenue_by_month(df_clean),
revenue_total, revenue_by_month

#fig, ax = plt.subplots(figsize=(9,4))
#revenue_monthly_bar(ax, revenue_over_time["month"], revenue_total["revenue"], "Revenue by month", "Month", "Revenue")

c:\Users\Mauro\Desktop\Gruppuppgift -Linnéa branch\Gruppuppgift\src\metrics.py:52: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x["revenue"].sum() / x["order_id"].nunique())
c:\Users\Mauro\Desktop\Gruppuppgift -Linnéa branch\Gruppuppgift\src\metrics.py:63: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x["revenue"].sum() / x["order_id"].nunique())
c:\Users\Mauro\Desktop\Gruppuppgift 

ValueError: too many values to unpack (expected 2)

### 4. Hur ser en typisk order ut? – AOV (Average Order Value) och spridning.

**Nyckeltal:** AOV (Average Order Value)

### 5. Topp-listor – topp-3 kategorier efter intäkt.

**Nyckeltal:** Top-3 kategorier efter intäkt

### 6. Eventuella avvikelser – något oväntat mönster som sticker ut?

*Insikter om avvikelser som går att utläsa från sammanställning av datan ovan*

## Rekommendationer

Avsluta rapporten med 2–3 korta rekommendationer (punktlista) baserat på era resultat, t.ex. “Satsa på kategori X i stad Y under månad Z”.